# STIL Integration

[STIL](http://www.star.bristol.ac.uk/~mbt/stil/), the Starlink Tables Infrastructure Library, is a Java API for working with astronomical data, including VOTable, FITS, SQL, ASCII, CSV, CDF, and GBIN formats.  This notebook shows how to load STIL, and configure BeakerX to display STIL StarTables with the BeakerX interactive table widget.

In [ ]:
%classpath add mvn commons-io commons-io 2.6

import org.apache.commons.io.FileUtils

stilUrl = "http://www.star.bristol.ac.uk/~mbt/stil/stil.jar"
stilFile = System.getProperty("java.io.tmpdir") + "/stilFiles/stil.jar"
FileUtils.copyURLToFile(new URL(stilUrl), new File(stilFile));

%classpath add dynamic stilFile

In [ ]:
import uk.ac.starlink.table.formats.StreamStarTable
import jupyter.Displayer
import jupyter.Displayers

Displayers.register(StreamStarTable.class, new Displayer<StreamStarTable>() {
    
    def getArray(t){
        nCol = t.getColumnCount()
        rseq = t.getRowSequence()
        tableAsArray = []
        while ( rseq.next() ) {
            row = rseq.getRow();
            tableAsArray.add(row)
        }
        rseq.close();
        tableAsArray.toArray()
    };  
        
    def getColumnNames(t){
        names = []
        nCol = t.getColumnCount();
        for ( int icol = 0; icol < nCol; icol++ ) {
            names.add(t.getColumnInfo(icol).getName())
        }
        names   
    }
    
    @Override
    public Map<String, String> display(StreamStarTable table) {
        
        array =  getArray(table)  
        columnNames = getColumnNames(table)
        
        new TableDisplay(
                (int)table.getRowCount(),
                (int)table.getColumnCount(),
                columnNames,
                new TableDisplay.Element() {
                  @Override
                  public String get(int columnIndex, int rowIndex) {
                    return array[rowIndex][columnIndex];
                  }
                }
        ).display();
        return OutputCell.DISPLAYER_HIDDEN;
      }
    });

In [ ]:
import org.apache.commons.io.FileUtils

messierUrl = "http://andromeda.star.bristol.ac.uk/data/messier.csv"
messierFile = System.getProperty("java.io.tmpdir") + "/stilFiles/messier.csv"
FileUtils.copyURLToFile(new URL(messierUrl), new File(messierFile));
"Done"

In [ ]:
import uk.ac.starlink.table.StarTableFactory

starTable= new StarTableFactory().makeStarTable( messierFile, "csv" );